In [1]:
from bedrock_agentcore_starter_toolkit import Runtime
from bedrock_agentcore_starter_toolkit.operations.runtime import destroy_bedrock_agentcore
from boto3.session import Session
from pathlib import Path
import os
import sys

# # Get the current notebook's directory
# current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

# print(current_dir)

# utils_dir = os.path.join(current_dir, '..')
# utils_dir = os.path.abspath(utils_dir)

# # Add to sys.path
# sys.path.insert(0, utils_dir)
# print("sys.path[0]:", sys.path[0])
from utils import setup_cognito_user_pool

In [2]:
boto_session = Session()
region = boto_session.region_name

# ssm_client = boto_session.client('ssm', region_name=region)
secrets_client = boto_session.client('secretsmanager', region_name=region)
agentcore_control_client = boto_session.client("bedrock-agentcore-control", region_name=region)
ssm_client = boto_session.client('ssm', region_name=region)

tool_name = "mcp_server_agentcore"

Creating MCP Server

In [6]:
%%writefile mcp_server.py
from mcp.server.fastmcp import FastMCP
from starlette.responses import JSONResponse

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

@mcp.tool()
def add_numbers(a: int, b: int) -> int:
    """Add two numbers together"""
    return a + b

@mcp.tool()
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers together"""
    return a * b

@mcp.tool()
def greet_user(name: str) -> str:
    """Greet a user by name"""
    return f"Hello, {name}! Nice to meet you."

if __name__ == "__main__":
    mcp.run(transport="streamable-http")

Writing mcp_server.py


Creating Local Testing Client

In [ ]:
%%writefile mcp_client.py
import asyncio
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    mcp_url = "http://localhost:8000/mcp"
    headers = {}

    async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
        read_stream,
        write_stream,
        _,
    ):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            tool_result = await session.list_tools()
            print("Available tools:")
            for tool in tool_result.tools:
                print(f"  - {tool.name}: {tool.description}")

if __name__ == "__main__":
    asyncio.run(main())

Writing my_mcp_client.py


Testing Locally
To test your MCP server locally:

Terminal 1: Start the MCP server

python mcp_server.py
Terminal 2: Run the test client

python my_mcp_client.py

Setting up Amazon Cognito for Authentication

In [5]:
print("Setting up Amazon Cognito user pool...")
cognito_config = setup_cognito_user_pool()
print("Cognito setup completed ✓")
print(f"User Pool ID: {cognito_config.get('pool_id', 'N/A')}")
print(f"Client ID: {cognito_config.get('client_id', 'N/A')}")

Setting up Amazon Cognito user pool...
Pool id: ap-southeast-2_ag1bQ4x1v
Discovery URL: https://cognito-idp.ap-southeast-2.amazonaws.com/ap-southeast-2_ag1bQ4x1v/.well-known/openid-configuration
Client ID: 2n4iv4pig4embpact494bg3m0a
Bearer Token: eyJraWQiOiJDXC9rSzN3elgzcFE2VWU2QnNYZmZnQnpYbkhZZkQ1WjBvTnFcL2F4ZkRRWjg9IiwiYWxnIjoiUlMyNTYifQ.eyJzdWIiOiIyOTFlOTRiOC1jMGYxLTcwZWMtMzIzYi02OTE4YWY1YjZkZDIiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAuYXAtc291dGhlYXN0LTIuYW1hem9uYXdzLmNvbVwvYXAtc291dGhlYXN0LTJfYWcxYlE0eDF2IiwiY2xpZW50X2lkIjoiMm40aXY0cGlnNGVtYnBhY3Q0OTRiZzNtMGEiLCJvcmlnaW5fanRpIjoiOGMxZGU1MWItOTdmOC00ZWUzLTg3OGMtZDkwNTBjMzQwYjU3IiwiZXZlbnRfaWQiOiJiMTI5ODYzMi1iYTRkLTQ3NzktOGI5Ni01ZWVjNDc5MzZmZmMiLCJ0b2tlbl91c2UiOiJhY2Nlc3MiLCJzY29wZSI6ImF3cy5jb2duaXRvLnNpZ25pbi51c2VyLmFkbWluIiwiYXV0aF90aW1lIjoxNzYzNzA2NzM4LCJleHAiOjE3NjM3MTAzMzgsImlhdCI6MTc2MzcwNjczOCwianRpIjoiN2RkMmY5NjEtM2FlNi00ZTY0LWEwYzAtMzE5NzQ0YTZiNjA1IiwidXNlcm5hbWUiOiJ0ZXN0dXNlciJ9.nSKBGn3ci31K-2yjMFVdDMmekuo9oE5tRhAUa3_CoUlzV

Configuring AgentCore Runtime Deployment

In [6]:
print(f"Using AWS region: {region}")

required_files = ['mcp_server.py', 'pyproject.toml']
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("All required files found ✓")

agentcore_runtime = Runtime()

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            cognito_config['client_id']
        ],
        "discoveryUrl": cognito_config['discovery_url'],
    }
}

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="mcp_server.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="pyproject.toml",
    region=region,
    authorizer_configuration=auth_config,
    protocol="MCP",
    agent_name=tool_name
)
print("Configuration completed ✓")

Entrypoint parsed: file=/home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/hosting-mcp-server-2/mcp_server.py, bedrock_agentcore_name=mcp_server
Configuring BedrockAgentCore agent: mcp_server_agentcore


Using AWS region: ap-southeast-2
All required files found ✓
Configuring AgentCore Runtime...


⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: /home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/hosting-mcp-server-2/Dockerfile
Generated .dockerignore: /home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/hosting-mcp-server-2/.dockerignore
Keeping 'mcp_server_agentcore' as default agent
Bedrock AgentCore configured: /home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/hosting-mcp-server-2/.bedrock_agentcore.yaml


Configuration completed ✓


Launching MCP Server to AgentCore Runtime

In [7]:
print("Launching MCP server to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch()
print("Launch completed ✓")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'mcp_server_agentcore' to account 381491838394 (ap-southeast-2)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: mcp_server_agentcore


Launching MCP server to AgentCore Runtime...
This may take several minutes...


✅ ECR repository available: 381491838394.dkr.ecr.ap-southeast-2.amazonaws.com/bedrock-agentcore-mcp_server_agentcore
Getting or creating execution role for agent: mcp_server_agentcore
Using AWS region: ap-southeast-2, account ID: 381491838394
Role name: AmazonBedrockAgentCoreSDKRuntime-ap-southeast-2-7f3ae149b4


✅ Reusing existing ECR repository: 381491838394.dkr.ecr.ap-southeast-2.amazonaws.com/bedrock-agentcore-mcp_server_agentcore


✅ Reusing existing execution role: arn:aws:iam::381491838394:role/AmazonBedrockAgentCoreSDKRuntime-ap-southeast-2-7f3ae149b4
✅ Execution role available: arn:aws:iam::381491838394:role/AmazonBedrockAgentCoreSDKRuntime-ap-southeast-2-7f3ae149b4
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: mcp_server_agentcore
Role name: AmazonBedrockAgentCoreSDKCodeBuild-ap-southeast-2-7f3ae149b4
Reusing existing CodeBuild execution role: arn:aws:iam::381491838394:role/AmazonBedrockAgentCoreSDKCodeBuild-ap-southeast-2-7f3ae149b4
Using .dockerignore with 44 patterns
Uploaded source to S3: mcp_server_agentcore/source.zip
Updated CodeBuild project: bedrock-agentcore-mcp_server_agentcore-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.1s
🔄 PROVISIONING started (total: 1s)
✅ PROVISIONING completed in 7.5s
🔄 DOWNLOAD_SOURCE started (total: 9s)


Launch completed ✓
Agent ARN: arn:aws:bedrock-agentcore:ap-southeast-2:381491838394:runtime/mcp_server_agentcore-Lj3QYO7TMP
Agent ID: mcp_server_agentcore-Lj3QYO7TMP


Storing Configuration for Remote Access

In [8]:
import boto3
import json

ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    cognito_credentials_response = secrets_client.create_secret(
        Name='mcp_server/cognito/credentials',
        Description='Cognito credentials for MCP server',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials stored in Secrets Manager")
except secrets_client.exceptions.ResourceExistsException:
    secrets_client.update_secret(
        SecretId='mcp_server/cognito/credentials',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials updated in Secrets Manager")

agent_arn_response = ssm_client.put_parameter(
    Name='/mcp_server/runtime/agent_arn',
    Value=launch_result.agent_arn,
    Type='String',
    Description='Agent ARN for MCP server',
    Overwrite=True
)
print("✓ Agent ARN stored in Parameter Store")

print("\nConfiguration stored successfully!")
print(f"Agent ARN: {launch_result.agent_arn}")

✓ Cognito credentials updated in Secrets Manager
✓ Agent ARN stored in Parameter Store

Configuration stored successfully!
Agent ARN: arn:aws:bedrock-agentcore:ap-southeast-2:381491838394:runtime/mcp_server_agentcore-Lj3QYO7TMP


Creating Remote Testing Client

In [9]:
%%writefile mcp_client_remote.py
import asyncio
import boto3
import json
import sys
import base64
import time
from boto3.session import Session
from datetime import timedelta
import traceback

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

def get_refresh_token(client_id, refresh_token, region):
    """Refresh access token using refresh token"""
    cognito_client = boto3.client('cognito-idp', region_name=region)
    auth_response = cognito_client.initiate_auth(
        ClientId=client_id,
        AuthFlow='REFRESH_TOKEN_AUTH',
        AuthParameters={'REFRESH_TOKEN': refresh_token}
    )
    return auth_response['AuthenticationResult']['AccessToken']

def get_valid_token(bearer_token, client_id, refresh_token, region):
    """Check token expiry and refresh if needed"""
    try:
        payload = bearer_token.split('.')[1]
        payload += '=' * (4 - len(payload) % 4)
        decoded = json.loads(base64.b64decode(payload))
        
        current_time = int(time.time())
        if decoded['exp'] - current_time < 300:
            print("🔄 Token expiring soon, refreshing...")
            new_token = get_refresh_token(client_id, refresh_token, region)
            print("✓ Token refreshed successfully")
            return new_token
        
        return bearer_token
    except Exception as e:
        print("🔄 Invalid token, refreshing...", e)
        traceback.print_exc()
        return get_refresh_token(client_id, refresh_token, region)

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='mcp_server/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        refresh_token = parsed_secret['refresh_token']
        client_id = parsed_secret['client_id']
        print("✓ Retrieved credentials from Secrets Manager")
        
        # Validate and refresh token if needed
        bearer_token = get_valid_token(bearer_token, client_id, refresh_token, region)
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    if not agent_arn or not bearer_token:
        print("Error: AGENT_ARN or BEARER_TOKEN not retrieved properly")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\nConnecting to: {mcp_url}")
    print("Headers configured ✓")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")
                
                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}")
                    print(f"   Description: {tool.description}")
                    if hasattr(tool, 'inputSchema') and tool.inputSchema:
                        properties = tool.inputSchema.get('properties', {})
                        if properties:
                            print(f"   Parameters: {list(properties.keys())}")
                    print()
                
                print(f"✅ Successfully connected to MCP server!")
                print(f"Found {len(tool_result.tools)} tools available.")
                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

Overwriting mcp_client_remote.py


Testing Your Deployed MCP Server

In [10]:
print("Testing deployed MCP server...")
print("=" * 50)
!python mcp_client_remote.py

Testing deployed MCP server...
Using AWS region: ap-southeast-2
Retrieved Agent ARN: arn:aws:bedrock-agentcore:ap-southeast-2:381491838394:runtime/mcp_server_agentcore-Lj3QYO7TMP
✓ Retrieved credentials from Secrets Manager

Connecting to: https://bedrock-agentcore.ap-southeast-2.amazonaws.com/runtimes/arn%3Aaws%3Abedrock-agentcore%3Aap-southeast-2%3A381491838394%3Aruntime%2Fmcp_server_agentcore-Lj3QYO7TMP/invocations?qualifier=DEFAULT
Headers configured ✓

🔄 Initializing MCP session...
✓ MCP session initialized

🔄 Listing available tools...

📋 Available MCP Tools:
🔧 add_numbers
   Description: Add two numbers together
   Parameters: ['a', 'b']

🔧 multiply_numbers
   Description: Multiply two numbers together
   Parameters: ['a', 'b']

🔧 greet_user
   Description: Greet a user by name
   Parameters: ['name']

✅ Successfully connected to MCP server!
Found 3 tools available.


Invoking MCP Tools Remotely

In [11]:
%%writefile invoke_mcp_tools.py
import asyncio
import boto3
import json
import sys
import base64
import time
from boto3.session import Session
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

def get_refresh_token(client_id, refresh_token, region):
    """Refresh access token using refresh token"""
    cognito_client = boto3.client('cognito-idp', region_name=region)
    auth_response = cognito_client.initiate_auth(
        ClientId=client_id,
        AuthFlow='REFRESH_TOKEN_AUTH',
        AuthParameters={'REFRESH_TOKEN': refresh_token}
    )
    return auth_response['AuthenticationResult']['AccessToken']

def get_valid_token(bearer_token, client_id, refresh_token, region):
    """Check token expiry and refresh if needed"""
    try:
        payload = bearer_token.split('.')[1]
        payload += '=' * (4 - len(payload) % 4)
        decoded = json.loads(base64.b64decode(payload))
        
        current_time = int(time.time())
        if decoded['exp'] - current_time < 300:
            print("🔄 Token expiring soon, refreshing...")
            new_token = get_refresh_token(client_id, refresh_token, region)
            print("✓ Token refreshed successfully")
            return new_token
        
        return bearer_token
    except:
        print("🔄 Invalid token, refreshing...")
        return get_refresh_token(client_id, refresh_token, region)

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='mcp_server/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        refresh_token = parsed_secret['refresh_token']
        client_id = parsed_secret['client_id']
        print("✓ Retrieved credentials from Secrets Manager")
        
        # Validate and refresh token if needed
        bearer_token = get_valid_token(bearer_token, client_id, refresh_token, region)
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\nConnecting to: {mcp_url}")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")
                
                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}: {tool.description}")
                
                print("\n🧪 Testing MCP Tools:")
                print("=" * 50)
                
                try:
                    print("\n➕ Testing add_numbers(5, 3)...")
                    add_result = await session.call_tool(
                        name="add_numbers",
                        arguments={"a": 5, "b": 3}
                    )
                    print(f"   Result: {add_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                try:
                    print("\n✖️  Testing multiply_numbers(4, 7)...")
                    multiply_result = await session.call_tool(
                        name="multiply_numbers",
                        arguments={"a": 4, "b": 7}
                    )
                    print(f"   Result: {multiply_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                try:
                    print("\n👋 Testing greet_user('Alice')...")
                    greet_result = await session.call_tool(
                        name="greet_user",
                        arguments={"name": "Alice"}
                    )
                    print(f"   Result: {greet_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                print("\n✅ MCP tool testing completed!")
                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

Overwriting invoke_mcp_tools.py


Test Tool Invocation

In [12]:
print("Testing MCP tool invocation...")
print("=" * 50)
!python invoke_mcp_tools.py

Testing MCP tool invocation...
Using AWS region: ap-southeast-2
Retrieved Agent ARN: arn:aws:bedrock-agentcore:ap-southeast-2:381491838394:runtime/mcp_server_agentcore-Lj3QYO7TMP
✓ Retrieved credentials from Secrets Manager

Connecting to: https://bedrock-agentcore.ap-southeast-2.amazonaws.com/runtimes/arn%3Aaws%3Abedrock-agentcore%3Aap-southeast-2%3A381491838394%3Aruntime%2Fmcp_server_agentcore-Lj3QYO7TMP/invocations?qualifier=DEFAULT

🔄 Initializing MCP session...
✓ MCP session initialized

🔄 Listing available tools...

📋 Available MCP Tools:
🔧 add_numbers: Add two numbers together
🔧 multiply_numbers: Multiply two numbers together
🔧 greet_user: Greet a user by name

🧪 Testing MCP Tools:

➕ Testing add_numbers(5, 3)...
   Result: 8

✖️  Testing multiply_numbers(4, 7)...
   Result: 28

👋 Testing greet_user('Alice')...
   Result: Hello, Alice! Nice to meet you.

✅ MCP tool testing completed!


Cleanup (Optional)

In [3]:
print("🗑️  Starting cleanup process...")

try:
    ssm_client.delete_parameter(Name='/mcp_server/runtime/agent_arn')
    print("✓ Parameter Store parameter deleted")
except ssm_client.exceptions.ParameterNotFound:
    print("ℹ️  Parameter Store parameter not found")

try:
    secrets_client.delete_secret(
        SecretId='mcp_server/cognito/credentials',
        ForceDeleteWithoutRecovery=True
    )
    print("✓ Secrets Manager secret deleted")
except secrets_client.exceptions.ResourceNotFoundException:
    print("ℹ️  Secrets Manager secret not found")

print("\n✅ Cleanup completed successfully!")

🗑️  Starting cleanup process...
✓ Parameter Store parameter deleted
✓ Secrets Manager secret deleted

✅ Cleanup completed successfully!


In [4]:
destroy_bedrock_agentcore(
    config_path=Path(".bedrock_agentcore.yaml"),
    agent_name=tool_name,
    delete_ecr_repo=True
)

Starting destroy operation for agent: mcp_server_agentcore (dry_run=False, delete_ecr_repo=True)
Skipping deletion of DEFAULT endpoint
Deleted AgentCore agent: arn:aws:bedrock-agentcore:ap-southeast-2:381491838394:runtime/mcp_server_agentcore-Lj3QYO7TMP
Checking ECR repository: bedrock-agentcore-mcp_server_agentcore in region: ap-southeast-2
Deleted 2 ECR images from bedrock-agentcore-mcp_server_agentcore
Deleted ECR repository: bedrock-agentcore-mcp_server_agentcore
Deleted CodeBuild project: bedrock-agentcore-mcp_server_agentcore-builder
Deleted inline policy CodeBuildExecutionPolicy from role AmazonBedrockAgentCoreSDKCodeBuild-ap-southeast-2-7f3ae149b4
Deleted CodeBuild IAM role: AmazonBedrockAgentCoreSDKCodeBuild-ap-southeast-2-7f3ae149b4
Deleted IAM role: AmazonBedrockAgentCoreSDKRuntime-ap-southeast-2-7f3ae149b4
Removed agent configuration: mcp_server_agentcore
Cleared default agent (no agents remaining)
Removed configuration file: .bedrock_agentcore.yaml
Destroy operation comple

DestroyResult(agent_name='mcp_server_agentcore', resources_removed=['AgentCore agent: arn:aws:bedrock-agentcore:ap-southeast-2:381491838394:runtime/mcp_server_agentcore-Lj3QYO7TMP', 'ECR images: 2 images from bedrock-agentcore-mcp_server_agentcore', 'ECR repository: bedrock-agentcore-mcp_server_agentcore', 'CodeBuild project: bedrock-agentcore-mcp_server_agentcore-builder', 'Deleted CodeBuild IAM role: AmazonBedrockAgentCoreSDKCodeBuild-ap-southeast-2-7f3ae149b4', 'IAM execution role: AmazonBedrockAgentCoreSDKRuntime-ap-southeast-2-7f3ae149b4', 'Agent configuration: mcp_server_agentcore', 'Configuration file (no agents remaining)'], warnings=['DEFAULT endpoint cannot be explicitly deleted, skipping'], errors=[], dry_run=False)